In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

Following the tutorials on the <a href="https://hmmlearn.readthedocs.io/en/stable/#user-guide-table-of-contents">hmmlearn docs</a>

# Background

The HMM is a generative probabilistic model. Under this framework, we have a sequence of _observable_ $X$ variables generated by a sequence of _internal hidden states_ $Z$. As the name implies, hidden states are not observed directly. 

_Transitions_ between hidden states are assumed to have the form of a (first-order) Markov chain. The transitions can be specified by a start probability vector $\pi$ and a transition probability matrix $A$.

The _emission probability_ of an observable $x_t$ can be any distribution with parameters $\theta$ conditioned on the current hidden state $z_t$.

An HMM is fully specified by $\pi$, $A$, and $\theta$.

There are three fundamental problems for HMMs:
1. Given the model parameters and observed data, estimate the optimal sequence of hidden states.
2. Given the model parameters and observed data, calculate the model likelihood.
3. Given just the observed data, estimate the model parameters.

The first and second problem can be solved by the **Viterbi algorithm** and the **Forward-Backward algorithm**, respectively. The last one can be solved via an iterative EM algorithm, known as **Baum-Welch algorithm**.

# Basics

In [3]:
import numpy as np
from hmmlearn import hmm

%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [4]:
np.random.seed(42)

<IPython.core.display.Javascript object>

In [12]:
# HMM with Gaussian emissions and n_states=3
model = hmm.GaussianHMM(n_components=3, covariance_type="full")

# Init probabilities
model.startprob_ = np.array([0.6, 0.3, 0.1])

# Transition probability matrix
model.transmat_ = np.array([[0.7, 0.2, 0.1],
                            [0.3, 0.5, 0.2],
                            [0.3, 0.3, 0.4]])

# Gaussian emission parameters - n_features=2
model.means_ = np.array([[0.0, 0.0], [3.0, -3.0], [5.0, 10.0]])
model.covars_ = np.tile(np.identity(2), (3, 1, 1))

X, Z = model.sample(100)
print(X.shape, Z.shape)

(100, 2) (100,)


<IPython.core.display.Javascript object>

The transition probability matrix $A$ need not be ergodic (e.g. the sample statistics of a long random sample of a stochastic process converge in $L^2$ to the true statistics). For instance, consider a left-right HMM:

In [15]:
lr = hmm.GaussianHMM(n_components=3, covariance_type="diag",
                     init_params="cm", params="cmt")
lr.startprob_ = np.array([1.0, 0.0, 0.0])
lr.transmat_ = np.array([[0.5, 0.5, 0.0],
                         [0.0, 0.5, 0.5,],
                         [0.0, 0.0, 1.0]])

<IPython.core.display.Javascript object>

# Problem 3: Training HMM parameters and inferring the hidden states

The input is a matrix of concatenated sequences of observations (i.e. samples) along with the lengths of the sequences. Since the EM algorithm is gradient-based, it will generally get stuck in local optima, hence running `fit` with various initializations and selecting the highest scoring model is recommended.

The score of the model can be calculated by the `score` method.

The inferred optimal hidden states can be obtained by calling the `predict` method. The `predict` method can be specified with a decoder algorithm, currently the Viterbi algorithm and MAP estimation are supported.

In [17]:
remodel = hmm.GaussianHMM(n_components=3, covariance_type="full", n_iter=100)
remodel.fit(X)

GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
            covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
            min_covar=0.001, n_components=3, n_iter=100, params='stmc',
            random_state=None, startprob_prior=1.0, tol=0.01,
            transmat_prior=1.0, verbose=False)

<IPython.core.display.Javascript object>

In [19]:
Z2 = remodel.predict(X)
print(Z2.shape)

(100,)


<IPython.core.display.Javascript object>

In [34]:
print(f"Z: {Z}\nZ2:{Z2}")  # Z2 == ( (Z1 - 1) % 3 )

Z: [1 2 2 2 0 0 0 0 0 0 0 0 0 0 2 2 0 0 0 0 1 2 2 2 0 2 2 2 1 1 1 1 2 2 1 0 2
 0 2 2 1 0 1 1 1 1 0 0 1 1 1 0 2 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 1
 1 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 0 0 0 0]
Z2:[0 1 1 1 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 0 1 1 1 2 1 1 1 0 0 0 0 1 1 0 2 1
 2 1 1 0 2 0 0 0 0 2 2 0 0 0 2 1 0 2 2 2 2 2 0 0 2 2 2 0 2 2 2 2 2 2 2 0 0
 0 2 0 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 1 2 2 2 2]


<IPython.core.display.Javascript object>

## Monitoring convergence

The number of iterations of EM algorithm is bounded above by `n_iter`. Training stops either after `n_iter` iterations were performed or the change in score is smaller than a specified tolerance `tol`.

We can use the `monitor_` attribute to check convergence

In [35]:
remodel.monitor_

ConvergenceMonitor(
    history=[-357.7756318437624, -357.7690842513329],
    iter=9,
    n_iter=100,
    tol=0.01,
    verbose=False,
)

<IPython.core.display.Javascript object>

In [36]:
remodel.monitor_.converged

True

<IPython.core.display.Javascript object>

## Working with multiple sequences

In [37]:
X1 = [[0.5], [1.0], [-1.0], [0.42], [0.24]]
X2 = [[2.4], [4.2], [0.5], [-0.24]]

<IPython.core.display.Javascript object>

In [38]:
X = np.concatenate([X1, X2])  # Concat all sequences
lengths = [len(X1), len(X2)]  # Array of sequence lengths

<IPython.core.display.Javascript object>

In [39]:
hmm.GaussianHMM(n_components=3).fit(X, lengths)

Fitting a model with 14 free scalar parameters with only 9 data points will result in a degenerate solution.


GaussianHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
            covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
            min_covar=0.001, n_components=3, n_iter=10, params='stmc',
            random_state=None, startprob_prior=1.0, tol=0.01,
            transmat_prior=1.0, verbose=False)

<IPython.core.display.Javascript object>

# Persisting Models

In [54]:
import pickle
with open("filename.pkl", "wb") as file: pickle.dump(remodel, file)
with open("filename.pkl", "rb") as file: pickle.load(file)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/anaconda3/envs/wgsegment/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-54-b82a0a8d8d81>", line 2, in <module>
    with open("filename.pkl", "wb") as file: pickle.dump(remodel, file)
FileNotFoundError: [Errno 2] No such file or directory: 'filename.pkl'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/envs/wgsegment/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/envs/wgsegment/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1148, in get_records
    ret

FileNotFoundError: [Errno 2] No such file or directory: 'filename.pkl'

<IPython.core.display.Javascript object>